In [1]:
import pandas as pd
import tsfresh
from tsfresh.utilities.dataframe_functions import impute

import problem
from problem import turn_prediction_to_event_list

data = problem.get_train_data(path="../")
X_train: pd.DataFrame = data[0]
y_train: pd.DataFrame = data[1]

events = turn_prediction_to_event_list(y_train)


def extract_located_area(event_index, delta=70):
    start = pd.to_datetime(events[event_index].begin)
    end = pd.to_datetime(events[event_index].end)
    X_df = X_train[(start - pd.Timedelta(hours=delta)):(end + pd.Timedelta(hours=delta))]
    y = y_train[(start - pd.Timedelta(hours=delta)):(end + pd.Timedelta(hours=delta))]
    return X_df, y


def search_features(event_index, column_id, delta=70):
    print(f"[Feature Extraction] Looking for features at event {event_index} on column {column_id} with delta {delta}")
    X_df, y = extract_located_area(event_index, delta)
    y = y.to_numpy()
    print(f"                     * Window size is {X_df.shape[0]}")
    print(f"                     * Extracting features...")
    extracted_features = tsfresh.extract_features(X_df, column_id=column_id)
    print(f"                     * Imputing values...")
    extracted_features = impute(extracted_features)
    print(f"                     * Selecting features...")
    extracted_features = tsfresh.select_features(extracted_features, y)
    return extracted_features

In [2]:
features = search_features(20, column_id="Beta", delta=80)
features

[Feature Extraction] Looking for features at event 20 on column Beta with delta 80
                     * Window size is 1178
                     * Extracting features...


Feature Extraction: 100%|██████████| 20/20 [03:37<00:00, 10.87s/it]


                     * Imputing values...


C:\Users\frigi\anaconda3\lib\site-packages\tsfresh\utilities\dataframe_functions.py:198: RuntimeWarning: The columns ['B__mean_abs_change' 'B__mean_change' 'B__mean_second_derivative_central'
 ... 'RmsBob__permutation_entropy__dimension_7__tau_1'
 'RmsBob__query_similarity_count__query_None__threshold_0.0'
 'RmsBob__mean_n_absolute_max__number_of_maxima_7'] did not have any finite values. Filling with zeros.
  warnings.warn(


                     * Selecting features...
                     * Found:            Vx__fft_coefficient__attr_"abs"__coeff_0  \
0.003134                                 345.612457   
0.006443                                 348.783722   
0.007832                                 340.418091   
0.007900                                 339.811707   
0.008222                                 341.588104   
...                                             ...   
22.208883                                312.919922   
25.034016                                461.295959   
28.912888                                455.584412   
31.559370                                313.386414   
77.912021                                460.763947   

           Vx__fft_coefficient__attr_"real"__coeff_0  \
0.003134                                 -345.612457   
0.006443                                 -348.783722   
0.007832                                 -340.418091   
0.007900                                

,"Vx__fft_coefficient__attr_""abs""__coeff_0","Vx__fft_coefficient__attr_""real""__coeff_0","Vx__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","Vx__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","Vx__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)","Vx__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)",Vx__quantile__q_0.1,Vx__quantile__q_0.8,Vx__quantile__q_0.7,Vx__quantile__q_0.6,...,Vy__quantile__q_0.9,"Vy__cwt_coefficients__coeff_0__w_2__widths_(2, 5, 10, 20)","Vy__cwt_coefficients__coeff_0__w_10__widths_(2, 5, 10, 20)","Vy__cwt_coefficients__coeff_0__w_20__widths_(2, 5, 10, 20)","Vy__fft_coefficient__attr_""real""__coeff_0",Vy__quantile__q_0.4,Vy__quantile__q_0.8,"Vy__cwt_coefficients__coeff_0__w_5__widths_(2, 5, 10, 20)",Bx__benford_correlation,Na_nl__range_count__max_1__min_-1
0.003134,345.612457,-345.612457,-67.028004,-94.791913,-134.056009,-211.961161,-345.612457,-345.612457,-345.612457,-345.612457,...,41.173817,25.251549,11.292836,7.985241,41.173817,41.173817,41.173817,15.970482,-0.145280,1.0
0.006443,348.783722,-348.783722,-67.643039,-95.661703,-135.286078,-213.906071,-348.783722,-348.783722,-348.783722,-348.783722,...,25.008461,15.337475,6.859127,4.850135,25.008461,25.008461,25.008461,9.700271,0.062915,1.0
0.007832,340.418091,-340.418091,-66.020610,-93.367242,-132.041220,-208.775501,-340.418091,-340.418091,-340.418091,-340.418091,...,-1.872299,-1.148265,-0.513520,-0.363113,-1.872299,-1.872299,-1.872299,-0.726226,0.864123,1.0
0.007900,339.811707,-339.811707,-65.903008,-93.200928,-131.806016,-208.403610,-339.811707,-339.811707,-339.811707,-339.811707,...,-1.503281,-0.921949,-0.412308,-0.291546,-1.503281,-1.503281,-1.503281,-0.583092,0.864123,1.0
0.008222,341.588104,-341.588104,-66.247522,-93.688145,-132.495045,-209.493060,-341.588104,-341.588104,-341.588104,-341.588104,...,0.666332,0.408656,0.182756,0.129228,0.666332,0.666332,0.666332,0.258457,0.864123,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22.208883,312.919922,-312.919922,-60.687621,-85.825257,-121.375243,-191.911109,-312.919922,-312.919922,-312.919922,-312.919922,...,-2.933529,-1.799108,-0.804586,-0.568928,-2.933529,-2.933529,-2.933529,-1.137856,-0.200946,1.0
25.034016,461.295959,-461.295959,-89.463638,-126.520690,-178.927275,-282.908863,-461.295959,-461.295959,-461.295959,-461.295959,...,9.046588,5.548195,2.481228,1.754493,9.046588,9.046588,9.046588,3.508987,-0.200946,1.0
28.912888,455.584412,-455.584412,-88.355941,-124.954171,-176.711883,-279.406019,-455.584412,-455.584412,-455.584412,-455.584412,...,14.303595,8.772272,3.923079,2.774036,14.303595,14.303595,14.303595,5.548072,0.864123,1.0
31.559370,313.386414,-313.386414,-60.778093,-85.953203,-121.556185,-192.197205,-313.386414,-313.386414,-313.386414,-313.386414,...,-2.409170,-1.477524,-0.660769,-0.467234,-2.409170,-2.409170,-2.409170,-0.934468,0.864123,1.0
